In [1]:
import os 
os.chdir('../../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv
import utils.interpretable_functions as interpret
import utils.RiskSLIM as slim
import utils.stumps as stumps
import utils.fairness_functions as fairness
from utils.load_settings import load_settings

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import roc_auc_score

from pprint import pprint
from riskslim.helper_functions import load_data_from_csv, print_model

# restore saved variables
%store -r summary_property2_KY_interpret

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2


C:\Users\binha\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



### CART & EBM

In [2]:
### load data
data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/KY data/kentucky_data.csv").sort_values('person_id')
data = data.drop(['arnold_fta_raw','arnold_nca_raw','arnold_nvca_raw'], axis=1)
x = data.loc[:,:'current_violence20']
y = data['property_two_year'].values

In [3]:
#### CART
depth = [7,8,9,10]
cart_summary = interpret.CART(X=x,
                         Y=y,
                         depth=depth,
                         seed=816)

#### EBM
estimators = [40]
depth = [2]
learning_rate = [0.5]
ebm_summary = interpret.EBM(X=x,
                       Y=y,
                       learning_rate=learning_rate,
                       depth=depth,
                       estimators=estimators,
                       seed =816)

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:43: RuntimeWarning:

divide by zero encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:43: RuntimeWarning:

divide by zero encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:43: RuntimeWarning:

divide by zero encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binh

In [4]:
np.mean(cart_summary['holdout_test_auc']), np.mean(cart_summary['auc_diffs'])

(0.7839893439994444, 0.030730612747162024)

In [5]:
np.mean(ebm_summary['holdout_test_auc']), np.mean(ebm_summary['auc_diffs'])

(0.8000461789243019, 0.010556051677545985)

## Lasso Stumps

In [6]:
## load whole data
data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/KY data/kentucky_stumps.csv").sort_values('person_id')
original_data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/KY data/kentucky_data.csv").sort_values('person_id')
original_data = original_data.loc[:, ['person_id', 'age_at_current_charge', 'p_charges']]
data = pd.merge(original_data, data, on='person_id')
X_stumps, Y_stumps = data.loc[:,:'current_violence201'], data['property_two_year'].values
Y_stumps[Y_stumps == -1] = 0
cols = X_stumps.columns[5:]

## load train & test data
train_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/KY data/kentucky_train_stumps.csv")
test_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/KY data/kentucky_test_stumps.csv")
X_train_stumps, Y_train_stumps = train_stumps.loc[:,:'current_violence201'], train_stumps['property_two_year'].values
X_test_stumps, Y_test_stumps = test_stumps.loc[:,:'current_violence201'], test_stumps['property_two_year'].values
Y_train_stumps[Y_train_stumps == -1] = 0
Y_test_stumps[Y_test_stumps == -1] = 0

#### Nested Cross Validation

In [7]:
stump_summary = stumps.stump_cv(X = X_stumps, 
                                Y = Y_stumps, 
                                columns=cols, 
                                c_grid={'C': [0.001, 0.002]}, 
                                seed = 816)

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars



In [8]:
stump_summary['best_params'], np.mean(stump_summary['holdout_test_auc']), np.mean(stump_summary['auc_diffs'])

([{'C': 0.002}, {'C': 0.002}, {'C': 0.002}, {'C': 0.002}, {'C': 0.002}],
 0.8014362799589743,
 0.002497517038949848)

##### Best Stump Model

In [9]:
best_stump_model = stumps.stump_model(X_train_stumps, 
                                      Y_train_stumps, 
                                      X_test_stumps, 
                                      Y_test_stumps, 
                                      c=0.002, 
                                      columns=cols, 
                                      seed=816)
len(best_stump_model['features'])

25

##### Prediction Table

In [13]:
stumps.latex_stump_table(best_stump_model['coefs'], 
                         best_stump_model['features'], 
                         best_stump_model['intercept'], 
                         best_stump_model['dictionary'])

\begin{{tabular}}{{|l|r|r|}} \hline
1. sex1 & 0.1727056624597589 & +... \\ \hline
2. age_at_current_charge18 & 0.04597550126865515 & +... \\ \hline
3. age_at_current_charge19 & 0.1358992367394272 & +... \\ \hline
4. age_at_current_charge20 & 0.07927985075910683 & +... \\ \hline
5. age_at_current_charge21 & 0.03954324547008586 & +... \\ \hline
6. age_at_current_charge24 & 0.0405121354849025 & +... \\ \hline
7. age_at_current_charge27 & 0.040250539921292516 & +... \\ \hline
8. age_at_current_charge30 & 0.12624489679109688 & +... \\ \hline
9. age_at_current_charge35 & 0.1488085986434939 & +... \\ \hline
10. age_at_current_charge39 & 0.220474840546233 & +... \\ \hline
11. age_at_current_charge43 & 0.11123027294744349 & +... \\ \hline
12. p_arrest2 & 0.5759410524050513 & +... \\ \hline
13. p_arrest3 & 0.20417998238085963 & +... \\ \hline
14. p_arrest4 & 0.13403531554899253 & +... \\ \hline
15. p_arrest6 & 0.1556138825321608 & +... \\ \hline
16. p_felony1 & 0.2677480080335172 & +... \\ \hlin

## RiskSLIM

In [10]:
## load whole data
data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/KY data/kentucky_stumps.csv")
data = data.sort_values('person_id')
original_data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/KY data/kentucky_data.csv")
original_data = original_data.loc[:,['person_id', 'age_at_current_charge', 'p_charges']]

## load train & test data
train_data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/KY data/kentucky_train_stumps.csv")
test_data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/KY data/kentucky_test_stumps.csv")

In [11]:
best_stump_model = stumps.stump_model(X_train_stumps, 
                                      Y_train_stumps, 
                                      X_test_stumps, 
                                      Y_test_stumps, 
                                      c=0.001, 
                                      columns=cols, 
                                      seed=816)
len(best_stump_model['features'])

19

In [12]:
selected_features = best_stump_model['features']

In [13]:
### Subset features
if 'sex1' in selected_features:
    selected_features = ['property_two_year', 'person_id', 'screening_date', 'race'] + selected_features
    indicator = 1
else:
    selected_features = ['property_two_year', 'person_id', 'screening_date', 'race', 'sex1'] + selected_features
    indicator = 0

sub_data = data[selected_features]
sub_data = pd.merge(sub_data, original_data, on='person_id')
sub_X, sub_Y = sub_data.iloc[:,1:], sub_data.iloc[:,0].values
sub_X.insert(0, '(Intercept)', 1)

In [14]:
riskslim_summary = slim.risk_nested_cv(X=sub_X, 
                                       Y=sub_Y, 
                                       indicator = indicator,
                                       y_label='property_two_year', 
                                       max_coef=5, 
                                       max_coef_number=5, 
                                       max_runtime=1000,
                                       max_offset=100,
                                       c=1e-6, 
                                       seed=816)

setting c0 = 0.0 to ensure that intercept is not penalized
10/30/19 @ 11:44 AM | 243 rows in lookup table
10/30/19 @ 11:44 AM | ------------------------------------------------------------
10/30/19 @ 11:44 AM | runnning initialization procedure
10/30/19 @ 11:44 AM | ------------------------------------------------------------
10/30/19 @ 11:44 AM | CPA produced 2 cuts
10/30/19 @ 11:44 AM | running naive rounding on 7 solutions
10/30/19 @ 11:44 AM | best objective value: 0.3499
10/30/19 @ 11:44 AM | rounding produced 5 integer solutions
10/30/19 @ 11:44 AM | best objective value is 0.3669
10/30/19 @ 11:44 AM | running sequential rounding on 7 solutions
10/30/19 @ 11:44 AM | best objective value: 0.3499
10/30/19 @ 11:44 AM | sequential rounding produced 6 integer solutions
10/30/19 @ 11:44 AM | best objective value: 0.2717
10/30/19 @ 11:44 AM | polishing 11 solutions
10/30/19 @ 11:44 AM | best objective value: 0.2717
10/30/19 @ 11:44 AM | polishing produced 5 integer solutions
10/30/19 @ 

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
10/30/19 @ 11:44 AM | adding 507 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.0845.
Tried aggregator 1 time.
Reduced MIP has 40 rows, 42 columns, and 117 nonzeros.
Reduced MIP has 19 binaries, 21 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

  57666 17898        0.0812    10        0.0819        0.0791   550634    3.44%          rho_17 N  57666  57665     23
Elapsed time = 27.63 sec. (41285.42 ticks, tree = 6.32 MB, solutions = 7)
  58535 18149        0.0800    18        0.0819        0.0791   560762    3.44%           rho_1 U  58535  58527     19
  59450 18420        0.0791    17        0.0819        0.0791   571135    3.44%          rho_10 U  59450  59449     19
  60350 18686        cutoff              0.0819        0.0791   581367    3.44%          rho_11 U  60350  60314     17
  61300 19017        0.0809     3        0.0819        0.0791   591352    3.44%           rho_0 U  61300  61299     27
  62419 19460        0.0817     9        0.0819        0.0791   600637    3.44%           rho_7 N  62419  62418     29
  63442 19863        0.0801    27        0.0819        0.0791   610168    3.44%          rho_14 D  63442  63440     21
  64320 20196        0.0799     3        0.0819        0.0791   620211    3.44%          rho_

 150346  6159        cutoff              0.0817        0.0813  1198157    0.43%           rho_5 U 150346  80663     25
 153691  3456        cutoff              0.0817        0.0815  1209950    0.26%          rho_11 D 153691 111673     40
 157081   424        cutoff              0.0817        0.0817  1219340    0.03%           rho_6 U 157081 133822     29

User cuts applied:  1500

Root node processing (before b&c):
  Real time             =    0.17 sec. (2.43 ticks)
Sequential b&c:
  Real time             =   83.84 sec. (149485.00 ticks)
                          ------------
Total (root+branch&cut) =   84.02 sec. (149487.43 ticks)
setting c0 = 0.0 to ensure that intercept is not penalized
10/30/19 @ 11:45 AM | 243 rows in lookup table
10/30/19 @ 11:45 AM | ------------------------------------------------------------
10/30/19 @ 11:45 AM | runnning initialization procedure
10/30/19 @ 11:45 AM | ------------------------------------------------------------
10/30/19 @ 11:46 AM | CPA produc

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
10/30/19 @ 11:46 AM | adding 477 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.0843.
Tried aggregator 1 time.
Reduced MIP has 40 rows, 42 columns, and 117 nonzeros.
Reduced MIP has 19 binaries, 21 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

...

 142779 36913        0.0818    15        0.0824        0.0805  1165303    2.40%          rho_14 U 142779 142777     29
 145719 36942        cutoff              0.0824        0.0805  1184362    2.35%           rho_5 U 145719 145717     18
 148599 36964        0.0812    11        0.0824        0.0806  1202733    2.29%           rho_6 N 148599 121400     28
 151426 36855        0.0822     9        0.0824        0.0806  1221314    2.24%           rho_5 U 151426 151425     40
 154214 36713        0.0812    14        0.0824        0.0806  1239695    2.18%           rho_5 D 154214 154213     30
 157004 36501        cutoff              0.0824        0.0807  1257524    2.12%          rho_12 D 157004 157002     27
 159791 36371        0.0821    16        0.0824        0.0807  1275406    2.07%           rho_9 D 159791 159790     27
 162560 36193        cutoff              0.0824        0.0808  1293241    2.01%           rho_0 U 162560  66251     32
 165293 35928        cutoff              0.0824 

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
10/30/19 @ 12:49 PM | adding 514 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.0841.
Tried aggregator 1 time.
Reduced MIP has 40 rows, 42 columns, and 117 nonzeros.
Reduced MIP has 19 binaries, 21 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

Elapsed time = 26.64 sec. (41283.13 ticks, tree = 7.18 MB, solutions = 8)
  62950 21438        0.0818    15        0.0820        0.0792   577141    3.47%          rho_10 N  62950  56197     16
  64065 21905        0.0814    11        0.0820        0.0792   587010    3.47%          rho_13 N  64065  64064     31
  64968 22209        0.0804    14        0.0820        0.0792   597460    3.47%         alpha_6 U  64968  64963     25
  65948 22608        0.0802    10        0.0820        0.0792   607385    3.47%           rho_0 D  65948  65947     22
  67011 23066        cutoff              0.0820        0.0792   616758    3.47%           rho_5 U  67011  67009     33
  67981 23442        0.0807    18        0.0820        0.0792   627156    3.47%           rho_8 D  67981  67979     21
  68952 23825        cutoff              0.0820        0.0792   637174    3.47%           rho_1 U  68952  68868     25
  69952 24132        0.0807    19        0.0820        0.0792   647019    3.47%          rho_

 162802 37004        cutoff              0.0820        0.0802  1390281    2.20%           rho_5 U 162802 162801     30
 166250 36842        0.0815    11        0.0820        0.0803  1411868    2.14%           rho_5 D 166250 166249     37
 169557 36537        0.0810    14        0.0820        0.0803  1433957    2.08%           rho_7 N 169557 150277     33
 172859 36181        cutoff              0.0820        0.0804  1455384    2.01%          rho_10 D 172859 172857     32
 176136 35758        0.0813     6        0.0820        0.0804  1476960    1.95%           rho_3 D 176136 176134     43
 179465 35394        cutoff              0.0820        0.0805  1497699    1.89%           rho_9 N 179465 103118     26
 182780 34983        0.0812    12        0.0820        0.0805  1517864    1.83%           rho_7 N 182780  63415     27
Elapsed time = 89.61 sec. (168173.09 ticks, tree = 14.08 MB, solutions = 9)
 186074 34567        cutoff              0.0820        0.0806  1538468    1.77%          rh

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:43: RuntimeWarning:

divide by zero encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars



In [15]:
np.mean(riskslim_summary['train_auc']), np.mean(riskslim_summary['test_auc'])

(0.7738120025663155, 0.7736687960579406)

### Single RiskSLIM Model

In [16]:
selected_features = ["property_two_year"] + best_stump_model['features']
sub_train_data = train_data[selected_features]
sub_test_data = test_data[selected_features]

## split x 
sub_train_X = sub_train_data.iloc[:,1:]
sub_train_X.insert(0, '(Intercept)', 1)
cols = sub_train_X.columns.tolist()
sub_train_X = sub_train_X.values
sub_test_X = sub_test_data.iloc[:,1:].values

## split y
sub_train_Y = sub_train_data.iloc[:,0].values.reshape(-1,1)
sub_test_Y = sub_test_data.iloc[:,0].values.reshape(-1,1)

## sample weight
sample_weights = np.repeat(1, len(sub_train_Y))

In [17]:
new_train_data = {
    'X': sub_train_X,
    'Y': sub_train_Y,
    'variable_names': cols,
    'outcome_name': 'property_two_year',
    'sample_weights': sample_weights
}

In [18]:
model_info, mip_info, lcpa_info = slim.risk_slim(new_train_data, 
                                                 max_coefficient=5, 
                                                 max_L0_value=5, 
                                                 c0_value=1e-6, 
                                                 max_offset=100, 
                                                 max_runtime=1000)
print_model(model_info['solution'], new_train_data)

setting c0 = 0.0 to ensure that intercept is not penalized
10/30/19 @ 12:52 PM | 243 rows in lookup table
10/30/19 @ 12:52 PM | ------------------------------------------------------------
10/30/19 @ 12:52 PM | runnning initialization procedure
10/30/19 @ 12:52 PM | ------------------------------------------------------------
10/30/19 @ 12:52 PM | CPA produced 2 cuts
10/30/19 @ 12:52 PM | running naive rounding on 7 solutions
10/30/19 @ 12:52 PM | best objective value: 0.3468
10/30/19 @ 12:52 PM | rounding produced 5 integer solutions
10/30/19 @ 12:52 PM | best objective value is 0.3638
10/30/19 @ 12:52 PM | running sequential rounding on 7 solutions
10/30/19 @ 12:52 PM | best objective value: 0.3468
10/30/19 @ 12:52 PM | sequential rounding produced 5 integer solutions
10/30/19 @ 12:52 PM | best objective value: 0.2690
10/30/19 @ 12:52 PM | polishing 10 solutions
10/30/19 @ 12:52 PM | best objective value: 0.2690
10/30/19 @ 12:52 PM | polishing produced 5 integer solutions
10/30/19 @ 

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
10/30/19 @ 12:52 PM | adding 505 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.0828.
Tried aggregator 1 time.
Reduced MIP has 40 rows, 42 columns, and 117 nonzeros.
Reduced MIP has 19 binaries, 21 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

  69288 23245        0.0792    19        0.0807        0.0782   590611    3.13%          rho_18 U  69288  69281     16
Elapsed time = 26.44 sec. (41278.84 ticks, tree = 7.56 MB, solutions = 7)
  70296 23572        0.0782    19        0.0807        0.0782   601963    3.13%        alpha_13 D  70296  70295     19
  71364 23843        0.0804    14        0.0807        0.0782   612861    3.13%           rho_5 U  71364  71362     31
  72417 24130        0.0795    14        0.0807        0.0782   623573    3.13%        alpha_13 U  72417  72416     19
  73491 24408        0.0782    15        0.0807        0.0782   633999    3.13%         alpha_8 U  73491  73474     19
  74560 24653        0.0782    12        0.0807        0.0782   644528    3.13%           rho_6 U  74560  74558     19
  75593 24932        0.0796    19        0.0807        0.0782   655301    3.13%           rho_0 U  75593  75591     14
  76574 25249        cutoff              0.0807        0.0782   666934    3.13%           rho

 189999 34942        0.0802    16        0.0807        0.0794  1483196    1.62%           rho_7 U 189999 189998     26
 194100 34264        0.0807    16        0.0807        0.0795  1507277    1.56%           rho_9 U 194100 194098     25
 198179 33467        0.0800    14        0.0807        0.0795  1531364    1.49%           rho_7 U 198179 112458     26
 202400 32838        0.0806     2        0.0807        0.0796  1554818    1.43%           rho_7 D 202400 202399     32
 206575 31952        0.0807    22        0.0807        0.0796  1578229    1.37%          rho_12 N 206575  53365     18
 210887 30847        cutoff              0.0807        0.0797  1602245    1.30%           rho_6 D 210887 116518     22
 215146 29740        cutoff              0.0807        0.0797  1625721    1.23%           rho_5 D 215146 124680     19
Elapsed time = 87.45 sec. (168164.12 ticks, tree = 11.80 MB, solutions = 7)
 219353 28426        cutoff              0.0807        0.0798  1648082    1.16%           r

In [19]:
sub_train_X = sub_train_X[:,1:]
sub_train_Y[sub_train_Y == -1] = 0
sub_test_Y[sub_test_Y == -1] = 0

In [20]:
roc_auc_score(sub_train_Y, slim.riskslim_prediction(sub_train_X, np.array(cols), model_info).reshape(-1,1))

0.7734220149897528

In [21]:
roc_auc_score(sub_test_Y, slim.riskslim_prediction(sub_test_X, np.array(cols), model_info).reshape(-1,1))

0.7768261372193861

### Arnold PSA

In [22]:
### load data
data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/KY data/kentucky_data.csv").sort_values('person_id')
settings = load_settings()
for decoder_name, decoder_dict in settings['decoders'].items():
    data = data.replace({decoder_name: decoder_dict})  
    
# compute scaled arnold scores
for score in ['arnold_nca' , 'arnold_nvca', 'arnold_fta']:
    data[score] = data[score + "_raw"]
    data = data.replace({score: settings[score + '_scaler'] })
    if score != 'arnold_nvca':
        data[score] = data[score].astype(int)
X_arnold = data.loc[:,['arnold_nca', 'sex', 'race', 'person_id', 'screening_date', 'age_at_current_charge', 'p_charges']]
Y_arnold = data['property_two_year'].values

In [24]:
## set up cross validation
cv = KFold(n_splits=5,shuffle=True,random_state=816)
arnold_auc = []
race_auc = []
condition_pn = []
no_condition_pn = []

i = 0
for train, test in cv.split(X_arnold, Y_arnold):
    
    train_x, train_y = X_arnold.iloc[train], Y_arnold[train]
    test_x, test_y = X_arnold.iloc[test], Y_arnold[test]
    holdout_with_attrs = test_x.copy()
    
    ################################
    ## arnold_auc
    arnold_auc.append(roc_auc_score(test_y, test_x['arnold_nca'].values))
    
    ################################
    ## race_auc
    try:
        arnold_race_auc = fairness.fairness_in_auc(df = holdout_with_attrs,
                                                   probs = test_x['arnold_nca'],
                                                   labels = test_y)
        arnold_race_auc_final = arnold_race_auc.assign(fold_num = [i]*arnold_race_auc['Attribute'].count())
        race_auc.append(arnold_race_auc_final)
    except:
        pass
    
    ################################
    ## condition pn
    arnold_condition_pn = fairness.conditional_balance_positive_negative(df = holdout_with_attrs,
                                                                         probs = test_x['arnold_nca'],
                                                                         labels = test_y)
    arnold_condition_pn_final = arnold_condition_pn.assign(fold_num = [i]*arnold_condition_pn['Attribute'].count())
    condition_pn.append(arnold_condition_pn_final)
    
    ################################
    ## no condition pn
    arnold_no_condition_pn = fairness.balance_positive_negative(df = holdout_with_attrs,
                                                                probs = test_x['arnold_nca'],
                                                                labels = test_y)
    arnold_no_condition_pn_final = arnold_no_condition_pn.assign(fold_num = [i]*arnold_no_condition_pn['Attribute'].count())
    no_condition_pn.append(arnold_no_condition_pn_final)
    
    i += 1
    
## race_auc
arnold_race_auc = []
try:
    arnold_race_auc = pd.concat(race_auc, ignore_index=True)
    arnold_race_auc.sort_values(["fold_num", "Attribute"], inplace=True)
    arnold_race_auc = arnold_race_auc.reset_index(drop=True)
except:
    pass

## condition_pn
arnold_condition_pn = pd.concat(condition_pn, ignore_index=True)
arnold_condition_pn.sort_values(["fold_num", "Attribute"], inplace=True)
arnold_condition_pn = arnold_condition_pn.reset_index(drop=True)

## no_condition_pn
arnold_no_condition_pn = pd.concat(no_condition_pn, ignore_index=True)
arnold_no_condition_pn.sort_values(["fold_num", "Attribute"], inplace=True)
arnold_no_condition_pn = arnold_no_condition_pn.reset_index(drop=True)

### Results

In [26]:
#### save results
summary_property2_KY_interpret = {"CART": cart_summary,
                                 "EBM": ebm_summary, 
                                 'Lasso Stumps': stump_summary, 
                                 'RiskSLIM': riskslim_summary, 
                                 'Arnold PSA': arnold_auc}
%store summary_property2_KY_interpret

Stored 'summary_property2_KY_interpret' (dict)


In [27]:
results = [["CART", np.mean(cart_summary['holdout_test_auc']), np.mean(cart_summary['auc_diffs'])],
           ["EBM", np.mean(ebm_summary['holdout_test_auc']), np.mean(ebm_summary['auc_diffs'])], 
           ["Lasso Stumps", np.mean(stump_summary['holdout_test_auc']), np.mean(stump_summary['auc_diffs'])],
           ['RiskSLIM', np.mean(riskslim_summary['test_auc'])],
           ['Arnold PSA', round(np.mean(arnold_auc), 3)]]
results

[['CART', 0.7839893439994444, 0.030730612747162024],
 ['EBM', 0.8000461789243019, 0.010556051677545985],
 ['Lasso Stumps', 0.8014362799589743, 0.002497517038949848],
 ['RiskSLIM', 0.7736687960579406],
 ['Arnold PSA', 0.718]]

In [28]:
auc = [np.mean(cart_summary['holdout_test_auc']), 
       np.mean(ebm_summary['holdout_test_auc']), 
       np.mean(stump_summary['holdout_test_auc']), 
       np.mean(riskslim_summary['test_auc'])]

In [29]:
path = "C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/KY Results/Models/Two Year/"
results = [["Property", np.str((round(np.mean(cart_summary['holdout_test_auc']), 3))) + " (" + np.str(round(np.std(cart_summary['holdout_test_auc']), 3)) + ")", 
            np.str(round(np.mean(ebm_summary['holdout_test_auc']),3)) + " (" + np.str(round(np.std(ebm_summary['holdout_test_auc']), 3)) + ")", 
            np.str(round(np.mean(stump_summary['holdout_test_auc']),3)) + " (" + np.str(round(np.std(stump_summary['holdout_test_auc']), 3)) + ")",             
            np.str(round(np.mean(riskslim_summary['test_auc']),3)) + " (" + np.str(round(np.std(riskslim_summary['test_auc']), 3)) + ")", 
            round(np.max(auc) - np.min(auc), 3),
            np.str(round(np.mean(arnold_auc), 3)) + " (" + np.str(round(np.std(arnold_auc),3)) + ")"]]
with open(path + 'Interpretable Models Summary.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)

### confusion matrix

In [30]:
cart_confusion = cart_summary['confusion_matrix_stats']
ebm_confusion = ebm_summary['confusion_matrix_stats']
riskslim_confusion = riskslim_summary['confusion_matrix_stats']
stumps_confusion = stump_summary['confusion_matrix_stats']

cart_confusion.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/confusion/two-year/property/cart_confusion.csv', index=None,header=True)
ebm_confusion.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/confusion/two-year/property/ebm_confusion.csv', index=None,header=True)
riskslim_confusion.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/confusion/two-year/property/riskslim_confusion.csv', index=None,header=True)
stumps_confusion.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/confusion/two-year/property/stumps_confusion.csv', index=None,header=True)

### calibration matrix

In [31]:
cart_calibration = cart_summary['calibration_stats']
ebm_calibration = ebm_summary['calibration_stats']
riskslim_calibration = riskslim_summary['calibration_stats']
stumps_calibration = stump_summary['calibration_stats']

cart_calibration.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/calibration/two-year/property/cart_calibration.csv', index=None,header=True)
ebm_calibration.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/calibration/two-year/property/ebm_calibration.csv', index=None,header=True)
riskslim_calibration.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/calibration/two-year/property/riskslim_calibration.csv', index=None,header=True)
stumps_calibration.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/calibration/two-year/property/stumps_calibration.csv', index=None,header=True)

### race auc matrix

In [32]:
cart_race_auc = cart_summary['race_auc']
ebm_race_auc = ebm_summary['race_auc']
riskslim_race_auc = riskslim_summary['race_auc']
stumps_race_auc = stump_summary['race_auc']

cart_race_auc.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/race-auc/two-year/property/cart_race_auc.csv', index=None,header=True)
ebm_race_auc.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/race-auc/two-year/property/ebm_race_auc.csv', index=None,header=True)
riskslim_race_auc.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/race-auc/two-year/property/riskslim_race_auc.csv', index=None,header=True)
stumps_race_auc.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/race-auc/two-year/property/stumps_race_auc.csv', index=None,header=True)
arnold_race_auc.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/race-auc/two-year/property/arnold_race_auc.csv', index=None,header=True)

### condition pn

In [33]:
cart_condition_pn = cart_summary['condition_pn']
ebm_condition_pn = ebm_summary['condition_pn']
riskslim_condition_pn = riskslim_summary['condition_pn']
stumps_condition_pn = stump_summary['condition_pn']

cart_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/condition-pn/two-year/property/cart_condition_pn.csv', index=None,header=True)
ebm_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/condition-pn/two-year/property/ebm_condition_pn.csv', index=None,header=True)
riskslim_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/condition-pn/two-year/property/riskslim_condition_pn.csv', index=None,header=True)
stumps_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/condition-pn/two-year/property/stumps_condition_pn.csv', index=None,header=True)
arnold_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/condition-pn/two-year/property/arnold_conditon_pn.csv', index=None,header=True)

### no condition pn

In [34]:
cart_no_condition_pn = cart_summary['no_condition_pn']
ebm_no_condition_pn = ebm_summary['no_condition_pn']
riskslim_no_condition_pn = riskslim_summary['no_condition_pn']
stumps_no_condition_pn = stump_summary['no_condition_pn']

cart_no_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/no-condition-pn/two-year/property/cart_no_condition_pn.csv', index=None,header=True)
ebm_no_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/no-condition-pn/two-year/property/ebm_no_condition_pn.csv', index=None,header=True)
riskslim_no_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/no-condition-pn/two-year/property/riskslim_no_condition_pn.csv', index=None,header=True)
stumps_no_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/no-condition-pn/two-year/property/stumps_no_condition_pn.csv', index=None,header=True)
arnold_no_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/no-condition-pn/two-year/property/arnold_no_conditon_pn.csv', index=None,header=True)